# 1. Import necessary libraries for data handling and visualization.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
#import ee
#import geemap

# 2. The eSBAE function:

In [3]:
def calculate_areas(db_total, strata_column, categories_column, total_area, z_score):
    
    df_full = db_total.copy()
    df_interpreted = db_total[~db_total[categories_column].isna()]
    
    # get all attributes
    categories = df_interpreted[categories_column].unique()
    
    # get strata
    strata, d = df_interpreted[strata_column].unique(), {}
    print(categories)
    # create stats for each entry
    for category in categories:
        
        if str(category) == 'nan':
            continue
            
        print(f' Calculating stats for {category}')
        # create binary class column
        df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
        print(f'There are {df_interpreted[category].sum()} entries of {category} in {categories_column}.')
        
        # initialize variables for category <> check the catergories > def, deg, gain
        categories_area, se_total = 0, 0
        d2 = {}
        for stratum in strata:
                        
            if str(stratum) == 'nan':
                continue
            
            # subset to stratum
            df_stratum = df_full[df_full[strata_column] == stratum]
            
            # get area proportion for that stratum on full dataset
            proportion_strata = len(df_stratum)/len(df_full)

            # get stratum area
            stratum_area = proportion_strata * total_area

            # get proportion of forest change within strata from interpreted data
            proportion_category = len(
                df_interpreted[(df_interpreted[strata_column] == stratum) & (df_interpreted[category] == 1)]
            ) / len(
                df_interpreted[df_interpreted[strata_column] == stratum]
            )
            
            # get area from proportion and full area
            category_stratum_area = proportion_category * stratum_area

            # get error from interpreted data for full stratum area
            var = np.var(df_interpreted[category][df_interpreted[strata_column] == stratum])
            sd = np.sqrt(var)               
            n = len(df_interpreted[df_interpreted[strata_column] == stratum])
            se = sd/np.sqrt(n) * stratum_area
                          
            # add for totals
            categories_area += category_stratum_area
            se_total += se**2
            
            # add to dictionary
            d2[f'area_stratum_{stratum}'] = category_stratum_area
            d2[f'ci_stratum_{stratum}']=z_score*se
        
        d2['area_total'] = categories_area
        d2['MOE'] = z_score*np.sqrt(se_total)
        d2['MOE_perc'] =  d2['MOE'] / categories_area * 100
        d[category] = d2
    
    
    return pd.DataFrame.from_dict(d, orient='index')

# 3 Load and prepare interpreted data from CEO Validation 
#### COG: 1001 points [07/12/2023]
#### set 1 = 1001 pts + set 2 = 2000 pts [13/12/2023]

In [38]:
#CEO_pts = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set1-sample-data-2023-12-06.csv', delimiter=';')
#CEO_set1raw = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set1-sample-data-2023-12-13.csv', delimiter=',')
#CEO_set2raw = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set2-sample-data-2023-12-13.csv', delimiter=',')
CEO_set1raw = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set1-sample-data-2023-12-14.csv', delimiter=',')
CEO_set2raw = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set2-sample-data-2023-12-14.csv', delimiter=',')

In [41]:
len(CEO_set1raw) # = 1001

1002

In [42]:
len(CEO_set2raw) # = 2000

2000

In [43]:
CEO_set1raw.columns.to_list() 
#CEO_pts.head()

['plotid',
 'sampleid',
 'lon',
 'lat',
 'email',
 'flagged',
 'collection_time',
 'analysis_duration',
 'imagery_title',
 'imagery_attributions',
 'sample_geom',
 'pl_aspect',
 'pl_brightness_max',
 'pl_red_mean',
 'pl_greenness_max',
 'pl_stratum',
 'pl_cusum_confidence',
 'pl_cusum_change_date',
 'pl_nir_max',
 'pl_red_min',
 'pl_bfast_means',
 'pl_elevation',
 'pl_red_sd',
 'pl_cnc_1520',
 'pl_swir1_sd',
 'pl_index',
 'pl_tmf_sub',
 'pl_dw_tree_prob__min',
 'pl_bs_slope_max',
 'pl_red_max',
 'pl_images',
 'pl_dist',
 'pl_bfast_magnitude',
 'pl_ccdc_magnitude',
 'pl_simple_combined',
 'pl_tmf_defyear',
 'pl_greenness_mean',
 'pl_tmf_2019',
 'pl_swir2_sd',
 'pl_nir_sd',
 'pl_gfc_tc00',
 'pl_greenness_min',
 'pl_tmf_degyear',
 'pl_tmf_2020',
 'pl_tmf_2017',
 'pl_bs_slope_min',
 'pl_lang_tree_height',
 'pl_bs_slope_sd',
 'pl_dw_class_mode',
 'pl_swir1_mean',
 'pl_wetness_sd',
 'pl_bs_slope_mean',
 'pl_ccdc_change_date',
 'pl_dw_tree_prob_mean',
 'pl_nir_mean',
 'pl_dw_tree_prob__stddev

In [44]:
count_set1 = CEO_set1raw['forêt ou non-forêt en 2016?'].value_counts()
print(count_set1)

forêt ou non-forêt en 2016?
forêt        640
non-forêt    362
Name: count, dtype: int64


In [45]:
#comme tous les points du set 1 ont été interpretés:
CEO_set1 = CEO_set1raw

In [46]:
count_set2 = CEO_set2raw['forêt ou non-forêt en 2016?'].value_counts()
print(count_set2)

forêt ou non-forêt en 2016?
non-forêt    948
forêt        867
Name: count, dtype: int64


In [47]:
#CEO_set2 = CEO_set2raw[CEO_set2raw['forêt ou non-forêt en 2016?'] != 'NaN']
#CEO_set2 = CEO_set2raw['forêt ou non-forêt en 2016?'].notna()

select_set2 = (CEO_set2raw['forêt ou non-forêt en 2016?'] == 'forêt') | (CEO_set2raw['forêt ou non-forêt en 2016?'] == 'non-forêt')
CEO_set2 = CEO_set2raw[select_set2]

len(CEO_set2)

1815

## 3.1 Different data checks and harmonisations 

In [48]:
# vérification doublons set 1
duplicated = CEO_set1['plotid'].duplicated().any() 
if duplicated:
    print ("problem")
else:
    print ("all good")

problem


In [49]:
doublonsCEO_set1 = CEO_set1[CEO_set1.duplicated(subset='plotid', keep=False)]
print(doublonsCEO_set1)

      plotid  sampleid        lon       lat                      email  \
7     541957    541957  11.235013 -3.836062      menguekarel@gmail.com   
8     541957    541957  11.235013 -3.836062  ndandoularissa7@gmail.com   
940  1118193   1118193  18.092651  2.905684   amelie.arquero@gmail.com   
941  1118193   1118193  18.092651  2.905684         kenombou@gmail.com   

     flagged   collection_time analysis_duration        imagery_title  \
7      False  2023-11-23 08:44        132.6 secs                  NaN   
8      False  2023-11-23 09:08        331.0 secs                  NaN   
940    False  2023-12-13 21:38      19620.3 secs  Planet NICFI Public   
941    False  2023-12-06 15:07        227.2 secs  Planet NICFI Public   

     imagery_attributions  ... Type du changement 2 (1)  \
7                     NaN  ...                      NaN   
8                     NaN  ...                      NaN   
940                   NaN  ...                      NaN   
941                   NaN  

In [51]:
CEO_set1['doublon'] = 'no'
CEO_set1['doublon'] = np.where((CEO_set1['plotid'] == 541957) & (CEO_set1['email'] == 'ndandoularissa7@gmail.com'), 'yes', CEO_set1['doublon'])
CEO_set1['doublon'] = np.where((CEO_set1['plotid'] == 1118193) & (CEO_set1['email'] == 'kenombou@gmail.com'), 'yes', CEO_set1['doublon'])

count_values_db = CEO_set1['doublon'].value_counts()
print(count_values_db)

doublon
no     1000
yes       2
Name: count, dtype: int64


In [52]:
dfCEO_set1 = CEO_set1[CEO_set1['doublon'] == 'no']
len(dfCEO_set1)

1000

In [53]:
# vérification doublons set 2
duplicated = CEO_set2['plotid'].duplicated().any() 
if duplicated:
    print ("problem")
else:
    print ("all good")

all good


In [54]:
CEO_set1['source'] = 'set1'
CEO_set2['source'] = 'set2'

/tmp/ipykernel_447/4156969318.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CEO_set2['source'] = 'set2'


In [55]:
CEO_cols = [ 'plotid',
 'forêt ou non-forêt en 2016?',
 'Type de non-forêt en 2016',
 'Type de forêt en 2016',
 'Y-a t-il un changement négatif sur la période 2016-2022',
 'Quel type de changement ? ',
 "Indiquez l'année du changement 1",
 'Type de moteur pour changement 1',
 'Le feu a t-il causé le changement ?',
 'Décrivez autres',
 'y-a t-il un second changement ?',
 'Type du changement 2 (1)',
 'Type de moteur pour changement 2 (1)',
 'Le feu a t-il causé le changement ?.1',
 'Décrivez autres (1) (0)',
 'Y-a t-il eu régénération  ?',
 'Indiquez la date de la régénération',
 'Définir la strate en 2022 ',
 'Type de non-forêt en 2022',
 'Type de forêt en 2022',
 'Commentaires',
  'source']

In [56]:
# concatenation set 1 + set 2
dfCEOconcat = pd.concat([CEO_set1[CEO_cols], CEO_set2[CEO_cols]], axis=0, ignore_index=True)
len(dfCEOconcat) # = 2817

2817

In [57]:
# vérification doublons set 2
doublon = dfCEOconcat['plotid'].duplicated().any() 
if duplicated:
    print ("problem")
else:
    print ("all good")

all good


In [21]:
#activites = dfCEO_pts_clean['Activités '].value_counts()
#print(activites)

In [58]:
pivot1 = pd.pivot_table(dfCEOconcat,values='source',index=['forêt ou non-forêt en 2016?'],columns=['Type de non-forêt en 2016'],aggfunc="count")
pivot1

Type de non-forêt en 2016,zone baties,eau,je ne sais pas,prairie aquatique,savane arborée/arbustive,savane herbacée,sol nu végétation éparse,terres cultivées annuelles,terres cultivées permanentes
forêt ou non-forêt en 2016?,,,,,,,,,
non-forêt,12,59,1,58,435,574,30,125,16


In [59]:
pivot2 = pd.pivot_table(dfCEOconcat,values='source',index=['forêt ou non-forêt en 2016?'],columns=['Type de forêt en 2016'],aggfunc="count")
pivot2

Type de forêt en 2016,1 - forêt dense,10 - plantation forestière,3 - forêt secondaire,4- forêt claire,7 - forêt mangrove,8 - forêt marécageuse,9 - forêt galérie
forêt ou non-forêt en 2016?,,,,,,,
forêt,695,8,538,20,1,124,121


In [60]:
## définition des catégories au niveau 1 (IPCC) - 2016
dfCEOconcat['ipcc_lulc_2016'] = 'problem'
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['forêt ou non-forêt en 2016?'] == 'forêt'), 'Terres forestieres', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == ' zone baties'), 'Etablissement humain', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'eau '), 'Terres humides', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'je ne sais pas'), 'Autres terres', dfCEOconcat['ipcc_lulc_2016']) ### verifier
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'prairie aquatique'), 'Terres humides', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'savane arborée/arbustive'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'savane herbacée'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'sol nu végétation éparse'), 'Autres terres', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'terres cultivées annuelles'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'terres cultivées permanentes'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2016'])
#dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['forêt ou non-forêt en 2016?'] == 'NaN'), 'invalide', dfCEOconcat['ipcc_lulc_2016']) ### ne marche pas ...???

count1 = dfCEOconcat['ipcc_lulc_2016'].value_counts()
print(count1)

ipcc_lulc_2016
Terres forestieres      1507
Terres gramineennes     1009
Terres cultivees         141
Terres humides           117
Autres terres             31
Etablissement humain      12
Name: count, dtype: int64


In [61]:
pivot3 = pd.pivot_table(dfCEOconcat,values='plotid',index=['forêt ou non-forêt en 2016?'],columns=['ipcc_lulc_2016'],aggfunc="count")
pivot3

ipcc_lulc_2016,Autres terres,Etablissement humain,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides
forêt ou non-forêt en 2016?,,,,,,
forêt,NaN,NaN,NaN,1507.0,NaN,NaN
non-forêt,31.0,12.0,141.0,NaN,1009.0,117.0


In [62]:
## définition des catégories au niveau 2 (classes nationales) - 2016
dfCEOconcat['n2_lulc_2016'] = dfCEOconcat['Type de non-forêt en 2016']
dfCEOconcat.loc[dfCEOconcat['forêt ou non-forêt en 2016?'] == 'forêt', 'n2_lulc_2016'] = dfCEOconcat['Type de forêt en 2016']

In [63]:
pivot3b = pd.pivot_table(dfCEOconcat,values='plotid',index=['n2_lulc_2016'],columns=['ipcc_lulc_2016'],aggfunc="count")
pivot3b

ipcc_lulc_2016,Autres terres,Etablissement humain,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides
n2_lulc_2016,,,,,,
zone baties,NaN,12.0,NaN,NaN,NaN,NaN
1 - forêt dense,NaN,NaN,NaN,695.0,NaN,NaN
10 - plantation forestière,NaN,NaN,NaN,8.0,NaN,NaN
3 - forêt secondaire,NaN,NaN,NaN,538.0,NaN,NaN
4- forêt claire,NaN,NaN,NaN,20.0,NaN,NaN
7 - forêt mangrove,NaN,NaN,NaN,1.0,NaN,NaN
8 - forêt marécageuse,NaN,NaN,NaN,124.0,NaN,NaN
9 - forêt galérie,NaN,NaN,NaN,121.0,NaN,NaN
eau,NaN,NaN,NaN,NaN,NaN,59.0


In [64]:
pivot4 = pd.pivot_table(dfCEOconcat,values='plotid',index=['Définir la strate en 2022 '],columns=['Type de non-forêt en 2022'],aggfunc="count")
pivot4

Type de non-forêt en 2022,eau,je ne sais pas,prairie aquatique,savane arbustive/arborée,savane herbacée,sol nu végétation éparse,terres cultivées annuelles,terres cultivées permanentes,zone baties
Définir la strate en 2022,,,,,,,,,
non-forêt,60,1,58,422,590,48,149,16,12


In [65]:
pivot5 = pd.pivot_table(dfCEOconcat,values='plotid',index=['Définir la strate en 2022 '],columns=['Type de forêt en 2022'],aggfunc="count")
pivot5

Type de forêt en 2022,1 - forêt dense,10 - plantation forestière,3 - forêt secondaire,4- forêt claire,8 - forêt marécageuse,9 - forêt galérie
Définir la strate en 2022,,,,,,
forêt,660,7,531,20,124,119


In [66]:
## définition des catégories au niveau 1 (IPCC) - 2022
dfCEOconcat['ipcc_lulc_2022'] = 'problem'
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Définir la strate en 2022 '] == 'forêt'), 'Terres forestieres', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'zone baties'), 'Etablissement humain', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'eau'), 'Terres humides', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'je ne sais pas'), 'Autres terres', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'prairie aquatique'), 'Terres humides', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'savane arbustive/arborée'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'savane herbacée'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'sol nu végétation éparse'), 'Autres terres', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'terres cultivées annuelles'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'terres cultivées permanentes'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2022'])
#dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Définir la strate en 2022 '] == 'NaN'), 'invalide', dfCEOconcat['ipcc_lulc_2022']) ### ne marche pas ...???

count1 = dfCEOconcat['ipcc_lulc_2022'].value_counts()
print(count1)

ipcc_lulc_2022
Terres forestieres      1461
Terres gramineennes     1012
Terres cultivees         165
Terres humides           118
Autres terres             49
Etablissement humain      12
Name: count, dtype: int64


In [67]:
## définition des catégories au niveau 2 (classes nationales) - 2022
dfCEOconcat['n2_lulc_2022'] = dfCEOconcat['Type de non-forêt en 2022']
dfCEOconcat.loc[dfCEOconcat['Définir la strate en 2022 '] == 'forêt', 'n2_lulc_2022'] = dfCEOconcat['Type de forêt en 2022']

In [68]:
pivot5b = pd.pivot_table(dfCEOconcat,values='plotid',index=['n2_lulc_2022'],columns=['ipcc_lulc_2022'],aggfunc="count")
pivot5b

ipcc_lulc_2022,Autres terres,Etablissement humain,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides
n2_lulc_2022,,,,,,
1 - forêt dense,NaN,NaN,NaN,660.0,NaN,NaN
10 - plantation forestière,NaN,NaN,NaN,7.0,NaN,NaN
3 - forêt secondaire,NaN,NaN,NaN,531.0,NaN,NaN
4- forêt claire,NaN,NaN,NaN,20.0,NaN,NaN
8 - forêt marécageuse,NaN,NaN,NaN,124.0,NaN,NaN
9 - forêt galérie,NaN,NaN,NaN,119.0,NaN,NaN
eau,NaN,NaN,NaN,NaN,NaN,60.0
je ne sais pas,1.0,NaN,NaN,NaN,NaN,NaN
prairie aquatique,NaN,NaN,NaN,NaN,NaN,58.0


In [69]:
#matrice des transitions n1 ipcc
tmatrix_n1 = pd.pivot_table(dfCEOconcat,values='plotid',index=['ipcc_lulc_2016'],columns=['ipcc_lulc_2022'],aggfunc="count")
tmatrix_n1

ipcc_lulc_2022,Autres terres,Etablissement humain,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides
ipcc_lulc_2016,,,,,,
Autres terres,28.0,NaN,NaN,2.0,1.0,NaN
Etablissement humain,NaN,12.0,NaN,NaN,NaN,NaN
Terres cultivees,NaN,NaN,135.0,5.0,1.0,NaN
Terres forestieres,19.0,NaN,23.0,1449.0,16.0,NaN
Terres gramineennes,2.0,NaN,7.0,5.0,994.0,1.0
Terres humides,NaN,NaN,NaN,NaN,NaN,117.0


In [70]:
tmatrix_n2 = pd.pivot_table(dfCEOconcat,values='plotid',index=['ipcc_lulc_2016', 'n2_lulc_2016'],columns=['ipcc_lulc_2022', 'n2_lulc_2022'],aggfunc="count")
tmatrix_n2

ipcc_lulc_2022                                     Autres terres  \
n2_lulc_2022                                      je ne sais pas   
ipcc_lulc_2016       n2_lulc_2016                                  
Autres terres        je ne sais pas                          1.0   
                     sol nu végétation éparse                NaN   
Etablissement humain  zone baties                            NaN   
Terres cultivees     terres cultivées annuelles              NaN   
                     terres cultivées permanentes            NaN   
Terres forestieres   1 - forêt dense                         NaN   
                     10 - plantation forestière              NaN   
                     3 - forêt secondaire                    NaN   
                     4- forêt claire                         NaN   
                     7 - forêt mangrove                      NaN   
                     8 - forêt marécageuse                   NaN   
                     9 - forêt galérie                       NaN   
Terres gramineennes  savane arborée/arbustive                NaN   
                     savane herbacée                         NaN   
Terres humides       eau                                     NaN   
                     prairie aquatique                       NaN   

ipcc_lulc_2022                                                              \
n2_lulc_2022                                      sol nu végétation éparse   
ipcc_lulc_2016       n2_lulc_2016                                            
Autres terres        je ne sais pas                                    NaN   
                     sol nu végétation éparse                         27.0   
Etablissement humain  zone baties                                      NaN   
Terres cultivees     terres cultivées annuelles                        NaN   
                     terres cultivées permanentes                      NaN   
Terres forestieres   1 - forêt dense                                   5.0   
                     10 - plantation forestière                        1.0   
                     3 - forêt secondaire                             12.0   
                     4- forêt claire                                   NaN   
                     7 - forêt mangrove                                NaN   
                     8 - forêt marécageuse                             NaN   
                     9 - forêt galérie                                 1.0   
Terres gramineennes  savane arborée/arbustive                          1.0   
                     savane herbacée                                   1.0   
Terres humides       eau                                               NaN   
                     prairie aquatique                                 NaN   

ipcc_lulc_2022                                    Etablissement humain  \
n2_lulc_2022                                               zone baties   
ipcc_lulc_2016       n2_lulc_2016                                        
Autres terres        je ne sais pas                                NaN   
                     sol nu végétation éparse                      NaN   
Etablissement humain  zone baties                                 12.0   
Terres cultivees     terres cultivées annuelles                    NaN   
                     terres cultivées permanentes                  NaN   
Terres forestieres   1 - forêt dense                               NaN   
                     10 - plantation forestière                    NaN   
                     3 - forêt secondaire                          NaN   
                     4- forêt claire                               NaN   
                     7 - forêt mangrove                            NaN   
                     8 - forêt marécageuse                         NaN   
                     9 - forêt galérie                             NaN   
Terres gramineennes  savane arborée/arbustive                      NaN   
                     savane herbacée    

In [81]:
## définition des activités
dfCEOconcat['DA1622'] = 'problem'
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] == 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] != 'Terres forestieres'), 'Def', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] == 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] == 'Terres forestieres'), 'SF', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] != 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] != 'Terres forestieres'), 'SNF', dfCEOconcat['DA1622'])

#degradation
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '1 - forêt dense') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '8 - forêt marécageuse') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '9 - forêt galérie') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '3 - forêt secondaire') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire') & (dfCEOconcat['Quel type de changement ? '] == 'Dégradation'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '1 - forêt dense') & (dfCEOconcat['n2_lulc_2022'] == '1 - forêt dense') & (dfCEOconcat['Quel type de changement ? '] == 'Dégradation'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '9 - forêt galérie') & (dfCEOconcat['n2_lulc_2022'] == '9 - forêt galérie') & (dfCEOconcat['Quel type de changement ? '] == 'Dégradation'), 'Deg', dfCEOconcat['DA1622'])

#Gains
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] != 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] == 'Terres forestieres'), 'Gain', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '3 - forêt secondaire') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire') & (dfCEOconcat['Y-a t-il un changement négatif sur la période 2016-2022'] == 'Non'), 'Gain', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '10 - plantation forestière') & (dfCEOconcat['n2_lulc_2022'] == '10 - plantation forestière'), 'SFPl', dfCEOconcat['DA1622'])

count1 = dfCEOconcat['DA1622'].value_counts()
print(count1)

DA1622
SNF     1298
SF       916
Gain     458
Deg       80
Def       58
SFPl       7
Name: count, dtype: int64


In [82]:
##
temp = pd.pivot_table(dfCEOconcat,values='plotid',index=['n2_lulc_2016', 'n2_lulc_2022'],columns=['Quel type de changement ? ', 'DA1622'],aggfunc="count")
temp

Quel type de changement ?                               Déforestation        \
DA1622                                                             Def  Deg   
n2_lulc_2016               n2_lulc_2022                                       
1 - forêt dense            1 - forêt dense                         NaN  NaN   
                           3 - forêt secondaire                    NaN  1.0   
                           savane herbacée                         4.0  NaN   
                           sol nu végétation éparse                5.0  NaN   
                           terres cultivées annuelles              NaN  NaN   
10 - plantation forestière sol nu végétation éparse                NaN  NaN   
3 - forêt secondaire       3 - forêt secondaire                    NaN  NaN   
                           savane herbacée                         5.0  NaN   
                           sol nu végétation éparse               11.0  NaN   
                           terres cultivées annuelles             19.0  NaN   
                           savane arbustive/arborée                NaN  NaN   
                           terres cultivées permanentes            1.0  NaN   
9 - forêt galérie          sol nu végétation éparse                NaN  NaN   
                           9 - forêt galérie                       NaN  NaN   
savane arborée/arbustive   sol nu végétation éparse                NaN  NaN   
                           terres cultivées annuelles              NaN  NaN   
                           savane arbustive/arborée                NaN  NaN   
                           terres cultivées permanentes            NaN  NaN   
savane herbacée            savane herbacée                         NaN  NaN   
                           sol nu végétation éparse                NaN  NaN   
                           terres cultivées annuelles              NaN  NaN   
sol nu végétation éparse   sol nu végétation éparse                NaN  NaN   

Quel type de changement ?                                         Dégradation  \
DA1622                                                    SF  SNF         Def   
n2_lulc_2016               n2_lulc_2022                                         
1 - forêt dense            1 - forêt dense               1.0  NaN         NaN   
                           3 - forêt secondaire          NaN  NaN         NaN   
                           savane herbacée               NaN  NaN         1.0   
                           sol nu végétation éparse      NaN  NaN         NaN   
                           terres cultivées annuelles    NaN  NaN         1.0   
10 - plantation forestière sol nu végétation éparse      NaN  NaN         1.0   
3 - forêt secondaire       3 - forêt secondaire          2.0  NaN         NaN   
                           savane herbacée               NaN  NaN         4.0   
                           sol nu végétation éparse      NaN  NaN         1.0   
                           terres cultivées annuelles    NaN  NaN         NaN   
                           savane arbustive/arborée      NaN  NaN         1.0   
                           terres cultivées permanentes  NaN  NaN         NaN   
9 - forêt galérie          sol nu végétation éparse      NaN  NaN         1.0   
                           9 - forêt galérie             NaN  NaN         NaN   
savane arborée/arbustive   sol nu végétation éparse      NaN  NaN         NaN   
                           terres cultivées annuelles    NaN  NaN         NaN   
                           savane arbustive/arborée      NaN  NaN         NaN   
                           terres cultivées permanentes  NaN  NaN         NaN   
savane herbacée            savane herbacée               NaN  1.0         NaN   
                           sol nu végétation éparse      NaN  NaN         NaN   
                           terres cultivées annuelles    NaN  NaN         NaN   
sol nu végétation éparse   sol nu végétation éparse      NaN  NaN         NaN   

In [84]:
##
temp = pd.pivot_table(dfCEOconcat,values='plotid',index=['ipcc_lulc_2016', 'ipcc_lulc_2022'],columns=['Quel type de changement ? ', 'DA1622'],aggfunc="count")
temp

Quel type de changement ?               Déforestation                  \
DA1622                                             Def  Deg   SF  SNF   
ipcc_lulc_2016      ipcc_lulc_2022                                      
Autres terres       Autres terres                  NaN  NaN  NaN  NaN   
Terres forestieres  Autres terres                 16.0  NaN  NaN  NaN   
                    Terres cultivees              20.0  NaN  NaN  NaN   
                    Terres forestieres             NaN  1.0  3.0  NaN   
                    Terres gramineennes            9.0  NaN  NaN  NaN   
Terres gramineennes Autres terres                  NaN  NaN  NaN  NaN   
                    Terres cultivees               NaN  NaN  NaN  NaN   
                    Terres gramineennes            NaN  NaN  NaN  1.0   

Quel type de changement ?               Dégradation             
DA1622                                          Def   Deg  SNF  
ipcc_lulc_2016      ipcc_lulc_2022                              
Autres terres       Autres terres               NaN   NaN  1.0  
Terres forestieres  Autres terres               3.0   NaN  NaN  
                    Terres cultivees            1.0   NaN  NaN  
                    Terres forestieres          NaN  75.0  NaN  
                    Terres gramineennes         6.0   NaN  NaN  
Terres gramineennes Autres terres               NaN   NaN  2.0  
                    Terres cultivees            NaN   NaN  6.0  
                    Terres gramineennes         NaN   NaN  4.0

In [85]:
len(dfCEOconcat)

2817

In [86]:
#dfCEOconcat.to_csv('/home/sepal-user/eSBAE_COG/output/dfCEOconcat.csv',index=False) 

#### 4. load national grid

In [87]:
#ee.Initialize()
#df_COG_grid = ee.FeatureCollection("users/andreasvollrath/COGSamples")

## scp -P 443 -r arquero@ssh.sepal.io:/home/sepal-user/module_results/esbae/my_first_esbae_project_congo_v1/COG_esbae_2015_2022_model.csv/ . (demander mdp à Amélie)
df_COG_grid = pd.read_csv('/home/sepal-user/eSBAE_COG/data/COG_esbae_2015_2022_model.csv', delimiter=',')
#len(df_COG_grid) # = 291595

In [88]:
# List all columns 
df_COG_grid.columns.tolist()

['images',
 'mon_images',
 'bfast_change_date',
 'bfast_magnitude',
 'bfast_means',
 'cusum_change_date',
 'cusum_confidence',
 'cusum_magnitude',
 'red_mean',
 'red_sd',
 'red_min',
 'red_max',
 'nir_mean',
 'nir_sd',
 'nir_min',
 'nir_max',
 'swir1_mean',
 'swir1_sd',
 'swir1_min',
 'swir1_max',
 'swir2_mean',
 'swir2_sd',
 'swir2_min',
 'swir2_max',
 'ndfi_mean',
 'ndfi_sd',
 'ndfi_min',
 'ndfi_max',
 'brightness_mean',
 'brightness_sd',
 'brightness_min',
 'brightness_max',
 'greenness_mean',
 'greenness_sd',
 'greenness_min',
 'greenness_max',
 'wetness_mean',
 'wetness_sd',
 'wetness_min',
 'wetness_max',
 'bs_slope_mean',
 'bs_slope_sd',
 'bs_slope_max',
 'bs_slope_min',
 'ccdc_change_date',
 'ccdc_magnitude',
 'aspect',
 'dw_class_mode',
 'dw_tree_prob__max',
 'dw_tree_prob__min',
 'dw_tree_prob__stdDev',
 'dw_tree_prob_mean',
 'elevation',
 'esa_lc20',
 'esa_lc21',
 'esri_lc17',
 'esri_lc18',
 'esri_lc19',
 'esri_lc20',
 'esri_lc21',
 'gfc_gain',
 'gfc_loss',
 'gfc_lossyear',


## 5. FULL dataframe with national GRID + 1k interpreted points

In [89]:
df_COG_grid.dtypes

images                 int64
mon_images             int64
bfast_change_date    float64
bfast_magnitude      float64
bfast_means          float64
                      ...   
stratum                int64
kmeans                 int64
PLOTID                 int64
LON                  float64
LAT                  float64
Length: 87, dtype: object

In [90]:
dfCEOconcat.dtypes

plotid                                                       int64
forêt ou non-forêt en 2016?                                 object
Type de non-forêt en 2016                                   object
Type de forêt en 2016                                       object
Y-a t-il un changement négatif sur la période 2016-2022     object
Quel type de changement ?                                   object
Indiquez l'année du changement 1                           float64
Type de moteur pour changement 1                            object
Le feu a t-il causé le changement ?                         object
Décrivez autres                                             object
y-a t-il un second changement ?                             object
Type du changement 2 (1)                                    object
Type de moteur pour changement 2 (1)                        object
Le feu a t-il causé le changement ?.1                       object
Décrivez autres (1) (0)                                     ob

In [91]:
dfCEOconcat.rename(columns={'plotid':'PLOTID'}, inplace=True)

In [92]:
df_COG_esbae = df_COG_grid[['PLOTID', 'kmeans']].merge(dfCEOconcat[['PLOTID', 'DA1622']], how='left', on='PLOTID')
len(df_COG_esbae)

291597

In [93]:
activites2 = df_COG_esbae['DA1622'].value_counts()
print(activites2)

DA1622
SNF     1298
SF       916
Gain     458
Deg       80
Def       58
SFPl       7
Name: count, dtype: int64


##### Perform area calculation using the stratum column. In this case the column is called kmeans. Use the merge dataframe (national grid points + CEO validated points)

## 6. Run the eSBAE function

In [94]:
calculate_areas(db_total=df_COG_esbae, strata_column='kmeans', categories_column='DA1622', total_area=len(df_COG_esbae), z_score=1.645)

['SNF' 'SF' 'Gain' 'Deg' 'Def' 'SFPl']
 Calculating stats for SNF
There are 1298 entries of SNF in DA1622.
 Calculating stats for SF
There are 916 entries of SF in DA1622.
 Calculating stats for Gain
There are 458 entries of Gain in DA1622.
 Calculating stats for Deg
There are 80 entries of Deg in DA1622.
 Calculating stats for Def
There are 58 entries of Def in DA1622.
 Calculating stats for SFPl
There are 7 entries of SFPl in DA1622.


/tmp/ipykernel_447/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_447/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_447/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_stratum_3,ci_stratum_3,area_total,MOE,MOE_perc
SNF,38652.130277,1407.787023,42539.403442,5998.167974,8386.327174,555.784469,89577.860892,6186.176503,6.905921
SF,18313.859534,1293.289434,146918.495220,6701.665437,3499.984783,423.342791,168732.339537,6838.430836,4.052828
Gain,6873.582242,884.664143,16149.217973,3982.888467,6159.064130,516.921288,29181.864345,4112.570827,14.092900
Deg,564.951965,267.104983,393.883365,647.318396,1522.720652,294.665534,2481.555982,759.732818,30.615179
Def,141.237991,133.993018,0.000000,0.000000,1249.994565,268.848008,1391.232556,300.388715,21.591553
SFPl,141.237991,133.993018,0.000000,0.000000,90.908696,74.609676,232.146687,153.364705,66.063706
